## Make the velocity vehicle DATABASE

In [11]:
import configparser
import mysql.connector
config = configparser.ConfigParser()
config.read('db.properties')

db_config = {
    'host': config.get('mysql_host', 'host'),
    'user': config.get('mysql_host', 'user'),
    'password': config.get('mysql_host', 'password'),
    'database': config.get('mysql_host', 'db') # Ensure you connect to the specific database
}

In [12]:
import pandas as pd

In [13]:
db_config

{'host': 'sql12.freesqldatabase.com',
 'user': 'sql12787715',
 'password': '6VL9teBf1t',
 'database': 'sql12787715'}

In [14]:
# Connect to MySQL
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()
cursor.execute(f"USE {db_config['database']}")

### make the car_brands table

In [15]:
# Drop the table if it exists
cursor.execute("DROP TABLE IF EXISTS car_brands")

# Create the car_brands table
cursor.execute("""
    CREATE TABLE car_brands (
        brand_name VARCHAR(50) NOT NULL,
        brand_id VARCHAR(255) PRIMARY KEY
    )
""")

In [16]:
import pandas as pd

# Read data from car_brands.xlsx and insert into car_brands table
try:
    df = pd.read_excel('.\\data\\car_brands.xlsx')
    for _, row in df.iterrows():
        try:
            cursor.execute(
                "INSERT INTO car_brands (brand_name, brand_id) VALUES (%s, %s)",
                (row['make'], row['brand_id'])
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")

### make the dealers table

In [17]:
cursor.execute("DROP TABLE IF EXISTS dealers")

# Create the dealers table
cursor.execute("""
    CREATE TABLE dealers (
        name VARCHAR(100) NOT NULL,
        brand_id VARCHAR(255),
        password VARCHAR(255) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        phone VARCHAR(10) NOT NULL UNIQUE,
        address VARCHAR(255) NOT NULL,
        city VARCHAR(100) NOT NULL,
        pincode INT NOT NULL,
        age INT NOT NULL check ((age<=70) and (age>=18)),
        gender VARCHAR(10) NOT NULL,
        description TEXT NOT NULL,
        username VARCHAR(100) PRIMARY KEY,
        FOREIGN KEY (brand_id) REFERENCES car_brands(brand_id)
    )
""")

In [18]:
# Read data from dealers.xlsx and insert into dealers table
try:
    df_dealers = pd.read_excel('.\\data\\dealers.xlsx')
    for _, row in df_dealers.iterrows():
        try:
            cursor.execute(
                """
                INSERT INTO dealers (
                    name, brand_id, password, email, phone, address, city, pincode, age, gender, description, username
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row['Name'],
                    row['Brand_ID'],
                    row['password'],
                    row['Email'],
                    row['Phone'],
                    row['Address'],
                    row['City'],
                    int(row['pincode']),
                    int(row['Age']),
                    row['gender'],
                    row['description'],
                    row['userName']
                )
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


### make the Customer table

In [19]:
# Drop the table if it exists
cursor.execute("DROP TABLE IF EXISTS customers")

# Create the customers table
cursor.execute("""
    CREATE TABLE customers (
        name VARCHAR(100) NOT NULL,
        password VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        phone VARCHAR(10) NOT NULL UNIQUE,
        address VARCHAR(255) NOT NULL,
        city VARCHAR(100) NOT NULL,
        pincode INT NOT NULL,
        gender VARCHAR(10) NOT NULL,
        age INT check ((age<=70) and (age>=18)),
        username VARCHAR(100) PRIMARY KEY
    )
""")

In [20]:
# Read data from customers.xlsx and insert into customers table
try:
    df_customers = pd.read_excel('.\\data\\customers.xlsx')
    for _, row in df_customers.iterrows():
        try:
            cursor.execute(
                """
                INSERT INTO customers (
                    name, password, email, phone, address, city, pincode, gender, age, username
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row['Name'],
                    row['password'],
                    row['Email'],
                    str(row['Phone']),
                    row['Address'],
                    row['City'],
                    int(row['Pincode']),
                    row['Gender'],
                    int(row['Age']),
                    row['userName']
                )
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


### cars table


In [ ]:
# Drop the cars table if it exists
cursor.execute("DROP TABLE IF EXISTS cars")

# Create the cars table with boolean columns for the specified fields
cursor.execute("""
    CREATE TABLE cars (
        car_id VARCHAR(255) PRIMARY KEY,
        brand_id VARCHAR(255),
        dealer_username VARCHAR(100),
        model VARCHAR(100) NOT NULL,
        variant VARCHAR(100) NOT NULL,
        rating INT,
        price_inr DECIMAL(15, 2) NOT NULL,
        engine_cc INT NOT NULL,
        cylinders INT NOT NULL CHECK ((cylinders>=2) AND (cylinders<=8)),
        fuel_capacity INT NOT NULL,
        fuel_type VARCHAR(100) NOT NULL,
        body_type VARCHAR(100) NOT NULL,
        mileage FLOAT NOT NULL CHECK ((mileage>=7) AND (mileage<=45)),
        gears INT NOT NULL CHECK ((gears>=4) AND (gears<=9)),
        steering_type VARCHAR(100) NOT NULL,
        keyless_entry BOOLEAN NOT NULL,
        power_hp FLOAT NOT NULL,
        torque_nm FLOAT NOT NULL,
        seating_capacity INT NOT NULL CHECK ((seating_capacity>=2) AND (seating_capacity<=16)),
        transmission_type VARCHAR(100) NOT NULL,
        warranty INT NOT NULL,
        sun_visor BOOLEAN NOT NULL,
        ventilation_system BOOLEAN NOT NULL,
        abs BOOLEAN NOT NULL,
        airbags BOOLEAN NOT NULL,
        colour VARCHAR(100) NOT NULL,
        FOREIGN KEY (brand_id) REFERENCES car_brands(brand_id),
        FOREIGN KEY (dealer_username) REFERENCES dealers(username)
    )
""")

In [22]:
# Read data from cars.xlsx and insert into cars table
try:
    df_cars = pd.read_excel('.\\data\\cars.xlsx')
    for _, row in df_cars.iterrows():
        try:
            cursor.execute(
                """
                INSERT INTO cars (
                    car_id, brand_id, dealer_username, model, variant,rating, price_inr, engine_cc, cylinders,
                    fuel_capacity, fuel_type, body_type, mileage, gears, power_steering, keyless_entry,
                    power_hp, torque_nm, seating_capacity, transmission_type, warranty, sun_visor,
                    ventilation_system, abs, airbags, colour
                ) VALUES (
                    %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row['Car_id'],
                    row['Brand_ID'],
                    row['DealerUsername'],
                    row['model'],
                    row['variant'],
                    row['rating'],
                    float(row['Price_INR']),
                    int(row['engine_cc']),
                    int(row['Cylinders']),
                    int(row['fuel_capacity']),
                    row['Fuel_type'],
                    row['Body_type'],
                    float(row['mileage']),
                    int(row['Gears']),
                    row['Power_steering'],
                    str(row['Keyless_entry']).lower() in ['true', '1', 'yes'],
                    float(row['power_hp']),
                    float(row['torque_nm']),
                    int(row['Seating_capacity']),
                    row['Transmission_type'],
                    int(row['warranty']),
                    str(row['Sun_visor']).lower() in ['true', '1', 'yes'],
                    str(row['Ventilation_system']).lower() in ['true', '1', 'yes'],
                    str(row['ABS']).lower() in ['true', '1', 'yes'],
                    str(row['Airbags']).lower() in ['true', '1', 'yes'],
                    row['color']
                )
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")

### make the sales table

In [23]:
# Drop the sales table if it exists
cursor.execute("DROP TABLE IF EXISTS sales")

# Create the sales table with the Date column and Car_ID as a foreign key to cars
cursor.execute("""
    CREATE TABLE sales (
        dealer_username VARCHAR(100),
        customer_username VARCHAR(100),
        brand_id VARCHAR(255),
        car_id VARCHAR(255),
        sales_id VARCHAR(20) PRIMARY KEY,
        date DATETIME,
        FOREIGN KEY (brand_id) REFERENCES car_brands(brand_id),
        FOREIGN KEY (dealer_username) REFERENCES dealers(username),
        FOREIGN KEY (customer_username) REFERENCES customers(username),
        FOREIGN KEY (car_id) REFERENCES cars(car_id)
    )
""")

In [24]:
try:
    df_sales = pd.read_excel('.\\data\\sales.xlsx')
    for _, row in df_sales.iterrows():
        # Check if brand_id exists in car_brands
        cursor.execute("SELECT 1 FROM car_brands WHERE brand_id = %s", (row['brandId'],))
        if cursor.fetchone() is None:
            print(f"Warning: brand_id '{row['brandId']}' does not exist in car_brands. Skipping row: {row.to_dict()}")
            continue
        # Check if car_id exists in cars
        cursor.execute("SELECT 1 FROM cars WHERE car_id = %s", (row['CarId'],))
        if cursor.fetchone() is None:
            print(f"Warning: car_id '{row['CarId']}' does not exist in cars. Skipping row: {row.to_dict()}")
            continue
        # Check if dealer_username exists in dealers
        cursor.execute("SELECT 1 FROM dealers WHERE username = %s", (row['DealerUsername'],))
        if cursor.fetchone() is None:
            print(f"Warning: dealer_username '{row['DealerUsername']}' does not exist in dealers. Skipping row: {row.to_dict()}")
            continue
        # Check if customer_username exists in customers
        cursor.execute("SELECT 1 FROM customers WHERE username = %s", (row['customerUsername'],))
        if cursor.fetchone() is None:
            print(f"Warning: customer_username '{row['customerUsername']}' does not exist in customers. Skipping row: {row.to_dict()}")
            continue
        try:
            cursor.execute(
                """
                INSERT INTO sales (
                    dealer_username, customer_username, brand_id, car_id, sales_id, date
                ) VALUES (
                    %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row['DealerUsername'],
                    row['customerUsername'],
                    row['brandId'],
                    row['CarId'],
                    row['sales_ID'],
                    row['sales_date']
                )
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


### add status column in the cars table

In [25]:
# 1. Add the status column to the cars table if it doesn't exist
try:
    cursor.execute("ALTER TABLE cars ADD COLUMN status VARCHAR(20) DEFAULT 'AVAILABLE' NOT NULL")
except Exception as e:
    # Ignore error if column already exists
    if "Duplicate column name" not in str(e):
        print(f"Error adding status column: {e}")

# 2. Set all cars to 'Available' by default
cursor.execute("UPDATE cars SET status = 'AVAILABLE'")

# 3. Update status to 'Sold' for cars present in the sales table
cursor.execute("UPDATE cars SET status = 'SOLD' WHERE car_id IN (SELECT car_id FROM sales)")

# 4. Commit the changes
conn.commit()
print("Status column updated in cars table.")

Status column updated in cars table.


### Some updates as per aditya said 

In [26]:
# Update all colour values in the cars table to upper case
cursor.execute("UPDATE cars SET colour = UPPER(colour)")
conn.commit()
print("All colours in the cars table have been converted to upper case.")

All colours in the cars table have been converted to upper case.


In [27]:
cursor.execute("alter table cars add column car_image varchar(255) default 'https://formbuilder.ccavenue.com/live/uploads/company_image/488/17316704336156_Event-Image-Not-Found.jpg' not null")
conn.commit()
print("Image URL column added to cars table with default value.")

Image URL column added to cars table with default value.


In [28]:
cursor.execute("alter table dealers add column profile_image varchar(255) default 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS59s6qBOFlkS5LN4Z0U3G71nCWWg3SuHGVMw&s' not null")
conn.commit()
print("Profile image column added to dealers table with default value.")

Profile image column added to dealers table with default value.


### Store the the dealer's imagee

In [31]:
# Read dealer images from Excel
dealer_images_df = pd.read_excel('cloudinary_data/dealer_images.xlsx')

# Ensure the profile_image column exists (if not, add it)
try:
    cursor.execute("""
        ALTER TABLE dealers 
        ADD COLUMN profile_image VARCHAR(255) 
        DEFAULT 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS59s6qBOFlkS5LN4Z0U3G71nCWWg3SuHGVMw&s' NOT NULL
    """)
    conn.commit()
except Exception as e:
    if "Duplicate column name" not in str(e):
        print(f"Error altering dealers table: {e}")

# Update each dealer's profile_image based on username
for _, row in dealer_images_df.iterrows():
    dealer_username = row['Image Name']
    image_url = row['Image URL']
    cursor.execute(
        "UPDATE dealers SET profile_image = %s WHERE username = %s",
        (image_url, dealer_username)
    )   
    conn.commit()
    print(f"Updated profile_image for dealer '{dealer_username}' to '{image_url}'")

Updated profile_image for dealer 'D_ANJALITOY001' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276554/dealer_images/D_ANJALITOY001.jpg'
Updated profile_image for dealer 'D_ARUNTAT001' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276555/dealer_images/D_ARUNTAT001.jpg'
Updated profile_image for dealer 'D_ARUNTAT002' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276556/dealer_images/D_ARUNTAT002.jpg'
Updated profile_image for dealer 'D_ASHATAT001' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276557/dealer_images/D_ASHATAT001.jpg'
Updated profile_image for dealer 'D_BALAJITAT001' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276557/dealer_images/D_BALAJITAT001.jpg'
Updated profile_image for dealer 'D_DEEPAKMAR001' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750276558/dealer_images/D_DEEPAKMAR001.jpg'
Updated profile_image for dealer 'D_DIVYATOY001' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v17502765

### Store the car images

In [32]:
# Read dealer images from Excel
car_images_df = pd.read_excel('cloudinary_data/car_images.xlsx')

# Ensure the profile_image column exists (if not, add it)
try:
    cursor.execute("""
        ALTER TABLE cars 
        ADD COLUMN car_image VARCHAR(255) 
        DEFAULT 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750308961/17316704336156_Event-Image-Not-Found_obotul.jpg' NOT NULL
    """)
    conn.commit()
except Exception as e:
    if "Duplicate column name" not in str(e):
        print(f"Error altering cars table: {e}")

# Update each car's image based on car id
for _, row in car_images_df.iterrows():
    car_id = row['Image Name']
    image_url = row['Image URL']
    cursor.execute(
        "UPDATE cars SET car_image = %s WHERE car_id = %s",
        (image_url, car_id)
    )   
    conn.commit()
    print(f"Updated car image for car '{car_id}' to '{image_url}'")

Updated car image for car '005F55F1-C804-4315-8BC9-46962DA53391' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750587119/car_images/005F55F1-C804-4315-8BC9-46962DA53391.png'
Updated car image for car '01C5B379-7BF7-43EB-AC17-5470CFECB6F9' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750587120/car_images/01C5B379-7BF7-43EB-AC17-5470CFECB6F9.png'
Updated car image for car '0234263E-D3E9-4590-9F09-F4AA9ED7690F' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750587121/car_images/0234263E-D3E9-4590-9F09-F4AA9ED7690F.png'
Updated car image for car '02CC8F43-A3DA-48B8-AE4C-DE9AAFA8350F' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750587122/car_images/02CC8F43-A3DA-48B8-AE4C-DE9AAFA8350F.png'
Updated car image for car '03CCECF9-4166-4500-896A-34E8D9910CD6' to 'https://res.cloudinary.com/dfecrk6wg/image/upload/v1750587123/car_images/03CCECF9-4166-4500-896A-34E8D9910CD6.png'
Updated car image for car '040DD762-191C-47C4-A5AD-9F485B7D62FC' to 'https://res

### make the password encrypted for the customer 

In [33]:
import bcrypt
import openpyxl
from openpyxl.styles import Font, Alignment

In [34]:
EXCEL_OUTPUT_FILE = './passwords/customers_password.xlsx'
CUSTOMER_TABLE = 'customers'
PASSWORD_STRENGTH = 10 # BCrypt cost factor
EXCLUDE_USER = "C_ADITYACHE001"

In [35]:
# --- BCrypt Hashing Functions ---
def hash_password_bcrypt(password: str, strength: int = PASSWORD_STRENGTH) -> bytes:
    """
    Hashes a password using BCrypt with a specified strength (cost factor).

    Args:
        password: The plain-text password string to hash.
        strength: The BCrypt cost factor (higher means more secure but slower).

    Returns:
        The hashed password as a bytes object.
    """
    password_bytes = password.encode('utf-8')
    hashed_password = bcrypt.hashpw(password_bytes, bcrypt.gensalt(rounds=strength))
    return hashed_password

In [36]:
def check_password_bcrypt(password: str, hashed_password: bytes) -> bool:
    """
    Checks if a plain-text password matches a given BCrypt hashed password.

    Args:
        password: The plain-text password string to check.
        hashed_password: The BCrypt hashed password (as bytes).

    Returns:
        True if the password matches the hash, False otherwise.
    """
    password_bytes = password.encode('utf-8')
    return bcrypt.checkpw(password_bytes, hashed_password)

In [37]:
try:
    cursor.execute(f"SELECT username, password FROM {CUSTOMER_TABLE}")
    customer_data = cursor.fetchall()

    if not customer_data:
        print(f"No customer data found in the '{CUSTOMER_TABLE}' table. Exiting.")

    original_passwords_for_excel = []
    updates_for_db = []

    # Prepare header for Excel sheet
    original_passwords_for_excel.append(["Username", "Original Password", "New Hashed Password"])

    print("\nProcessing customer passwords...")
    for username, original_password in customer_data:
        hashed_password_str = original_password # Default to original if not altered

        if username == EXCLUDE_USER:
            print(f"Skipping password alteration for '{username}' as requested.")
            # For Aditya Raj, the 'new hashed password' will simply be his original password
            # as it's not being changed.
        else:
            # Hash the password for other users
            hashed_password_bytes = hash_password_bcrypt(original_password)
            hashed_password_str = hashed_password_bytes.decode('utf-8') # Convert to string for DB and Excel
            updates_for_db.append((hashed_password_str, username))

            print(f"Password for '{username}' has been hashed.")

        # Append data to Excel row: username, original password, and the (potentially new) hashed password
        original_passwords_for_excel.append([username, original_password, hashed_password_str])

    # 2. Store original and new hashed passwords in an Excel sheet
    print(f"\nCreating Excel file: {EXCEL_OUTPUT_FILE} with original and new passwords...")
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Password Audit"

    # Write data to Excel
    for row_data in original_passwords_for_excel:
        sheet.append(row_data)

    # Style the header
    header_font = Font(bold=True)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = Alignment(horizontal='center')

    # Auto-fit column width
    for column in sheet.columns:
        max_length = 0
        column_letter = column[0].column_letter # Get the column letter
        for cell in column:
            try:
                if cell.value is not None and len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except Exception:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column_letter].width = adjusted_width

    workbook.save(EXCEL_OUTPUT_FILE)
    print(f"Password audit data saved to {EXCEL_OUTPUT_FILE}")

    # 3. Alter passwords in the SQL table with encrypted hash code
    if updates_for_db:
        print("\nUpdating passwords in the database...")
        for hashed_pwd, username in updates_for_db:
            # --- Debugging print statement: Check values before execution ---
            print(f"Attempting to update username: '{username}' with hashed password: '{hashed_pwd[:20]}...' (length: {len(hashed_pwd)})")
            # MySQL uses %s as a placeholder
            cursor.execute(f"UPDATE {CUSTOMER_TABLE} SET password = %s WHERE username = %s", (hashed_pwd, username))
        conn.commit()
        print("Database passwords updated successfully for all eligible users.")
    else:
        print("No passwords needed updating (perhaps only the excluded user was found, or no data).")

    print("\nScript completed successfully!")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


Processing customer passwords...
Password for 'C_AJAYTHO001' has been hashed.
Password for 'C_AMITABHERO001' has been hashed.
Password for 'C_ANITACOI001' has been hashed.
Password for 'C_ARJUNDIN001' has been hashed.
Password for 'C_DEEPATIR001' has been hashed.
Password for 'C_DIVYATHA001' has been hashed.
Password for 'C_HARISHCHE001' has been hashed.
Password for 'C_KIRANTIR001' has been hashed.
Password for 'C_LAKSHMIERO001' has been hashed.
Password for 'C_MEENAKAN001' has been hashed.
Password for 'C_NITINVEL001' has been hashed.
Password for 'C_PREETIVEL001' has been hashed.
Password for 'C_PRIYATIR001' has been hashed.
Password for 'C_RAHULSAL001' has been hashed.
Password for 'C_RINADIN001' has been hashed.
Password for 'C_ROHITMAD001' has been hashed.
Password for 'C_SAHILCHE001' has been hashed.
Password for 'C_SANDEEPTIR001' has been hashed.
Password for 'C_SNEHATHO001' has been hashed.
Password for 'C_SUNITASAL001' has been hashed.
Password for 'C_SWETHACOI001' has been 

In [38]:
EXCEL_OUTPUT_FILE = './passwords/dealers_password.xlsx'
DEALER_TABLE = 'dealers'
EXCLUDE_USER = "D_ANIKETFOR001"

In [39]:
try:
    cursor.execute(f"SELECT username, password FROM {DEALER_TABLE}")
    dealer_data = cursor.fetchall()

    if not dealer_data:
        print(f"No dealer data found in the '{DEALER_TABLE}' table. Exiting.")

    original_passwords_for_excel = []
    updates_for_db = []

    # Prepare header for Excel sheet
    original_passwords_for_excel.append(["Username", "Original Password", "New Hashed Password"])

    print("\nProcessing dealer passwords...")
    for username, original_password in dealer_data:
        hashed_password_str = original_password # Default to original if not altered

        if username == EXCLUDE_USER:
            print(f"Skipping password alteration for '{username}' as requested.")
            # For Aniket Pal, the 'new hashed password' will simply be his original password
            # as it's not being changed.
        else:
            # Hash the password for other users
            hashed_password_bytes = hash_password_bcrypt(original_password)
            hashed_password_str = hashed_password_bytes.decode('utf-8') # Convert to string for DB and Excel
            updates_for_db.append((hashed_password_str, username))

            print(f"Password for '{username}' has been hashed.")

        # Append data to Excel row: username, original password, and the (potentially new) hashed password
        original_passwords_for_excel.append([username, original_password, hashed_password_str])

    # 2. Store original and new hashed passwords in an Excel sheet
    print(f"\nCreating Excel file: {EXCEL_OUTPUT_FILE} with original and new passwords...")
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Password Audit"

    # Write data to Excel
    for row_data in original_passwords_for_excel:
        sheet.append(row_data)

    # Style the header
    header_font = Font(bold=True)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = Alignment(horizontal='center')

    # Auto-fit column width
    for column in sheet.columns:
        max_length = 0
        column_letter = column[0].column_letter # Get the column letter
        for cell in column:
            try:
                if cell.value is not None and len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except Exception:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column_letter].width = adjusted_width

    workbook.save(EXCEL_OUTPUT_FILE)
    print(f"Password audit data saved to {EXCEL_OUTPUT_FILE}")

    # 3. Alter passwords in the SQL table with encrypted hash code
    if updates_for_db:
        print("\nUpdating passwords in the database...")
        for hashed_pwd, username in updates_for_db:
            # --- Debugging print statement: Check values before execution ---
            print(f"Attempting to update username: '{username}' with hashed password: '{hashed_pwd[:20]}...' (length: {len(hashed_pwd)})")
            # MySQL uses %s as a placeholder
            cursor.execute(f"UPDATE {DEALER_TABLE} SET password = %s WHERE username = %s", (hashed_pwd, username))
        conn.commit()
        print("Database passwords updated successfully for all eligible users.")
    else:
        print("No passwords needed updating (perhaps only the excluded user was found, or no data).")

    print("\nScript completed successfully!")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


Processing dealer passwords...
Password for 'D_ANJALITOY001' has been hashed.
Password for 'D_ARUNTAT001' has been hashed.
Password for 'D_ARUNTAT002' has been hashed.
Password for 'D_ASHATAT001' has been hashed.
Password for 'D_BALAJITAT001' has been hashed.
Password for 'D_DEEPAKMAR001' has been hashed.
Password for 'D_DIVYATOY001' has been hashed.
Password for 'D_GOPINATHMER001' has been hashed.
Password for 'D_HARISHHON001' has been hashed.
Password for 'D_KARTHIKFOR001' has been hashed.
Password for 'D_KAVITHAHYU001' has been hashed.
Password for 'D_LAKSHMIFOR001' has been hashed.
Password for 'D_MANOJFOR001' has been hashed.
Password for 'D_MEERAMER001' has been hashed.
Password for 'D_NEHAMER001' has been hashed.
Password for 'D_POOJAMAR001' has been hashed.
Password for 'D_PRAKASHBMW001' has been hashed.
Password for 'D_PRIYAHYU001' has been hashed.
Password for 'D_RAJESHHYU001' has been hashed.
Password for 'D_RAMESHMAR001' has been hashed.
Password for 'D_REVATHIHON001' has 